<a href="https://colab.research.google.com/github/KAVYANSHTYAGI/Food-Image-Classifier/blob/main/novel_architecture_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau , LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score,classification_report
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

base_dir_training = "/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training"
class_labels_training = os.listdir(base_dir_training)
print(len(class_labels_training))

base_dir_testing = "/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/testing"
class_labels_testing = os.listdir(base_dir_testing)
print(len(class_labels_testing))

30
30


In [4]:
import os
import multiprocessing
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def process_images(args):
    """Function to load and preprocess images in a given directory"""
    path, count = args
    data = []

    for img in os.listdir(path):
        try:
            image = load_img(os.path.join(path, img), color_mode="rgb", target_size=(224, 224))
            image = img_to_array(image) / 255.0  # Normalize
            data.append([image, count])
        except Exception as e:
            pass  # Handle errors silently

    return data

def parallel_data_loader(base_dir, class_labels):
    """Function to parallelize data loading"""
    args = [(os.path.join(base_dir, label), idx) for idx, label in enumerate(class_labels)]

    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.map(process_images, args)

    # Flatten the list of lists into a single list
    return [item for sublist in results for item in sublist]

# Training data
data_training = parallel_data_loader(base_dir_training, class_labels_training)

# Testing data
data_testing = parallel_data_loader(base_dir_testing, class_labels_testing)


In [5]:
print(len(data_training))
print(len(data_testing))

num_classes = 30

8039
2027


In [6]:
X_train,y_train = zip(*data_training)

x_train = np.array(X_train)
y_train = np.array(y_train)


X_test,y_test = zip(*data_testing)

x_test = np.array(X_test)
y_test = np.array(y_test)

In [7]:
y_test = to_categorical(y_test, num_classes=num_classes)
y_train = to_categorical(y_train, num_classes=num_classes)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'               # Fill missing pixels after transformations
)


val_test_datagen = ImageDataGenerator()


train_generator = train_datagen.flow(x_train, y_train)
val_generator = val_test_datagen.flow(x_test, y_test)

In [ ]:
import tensorflow as tf
import numpy as np

def multi_input_dataset(generator, num_inputs):
    """
    Converts an ImageDataGenerator into a tf.data.Dataset that supports multiple inputs.

    Args:
    - generator: The ImageDataGenerator that yields (images, labels).
    - num_inputs: Number of model inputs.

    Returns:
    - A tf.data.Dataset that provides multiple inputs.
    """

    def _generator():
        for batch_images, batch_labels in generator:
            # Duplicate input for all three branches
            yield [batch_images] * num_inputs, batch_labels

    # ✅ Correct way to get the number of classes
    num_classes = len(generator.class_indices)

    # Create dataset
    dataset = tf.data.Dataset.from_generator(
        _generator,
        output_signature=(
            [tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)] * num_inputs,
            tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)  # ✅ Uses `len(generator.class_indices)`
        )
    )

    return dataset


In [ ]:
def prepare_multi_input(image, label):
    # Duplicate the same image for each branch
    return (image, image, image), label

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_multi_input = train_dataset.map(prepare_multi_input).batch(64).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_multi_input = val_dataset.map(prepare_multi_input).batch(64).prefetch(tf.data.AUTOTUNE)


In [ ]:
train_multi_input = None
val_multi_input = None

In [ ]:
# Get input shape dynamically
input_shape = (224, 224, 3)  # Ensure correct shape
num_classes = y_train.shape[-1]

# Define dataset correctly
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(x_train, y_train),
    output_signature=(
        (tf.TensorSpec(shape=input_shape, dtype=tf.float32),  # Input 1
         tf.TensorSpec(shape=input_shape, dtype=tf.float32),  # Input 2
         tf.TensorSpec(shape=input_shape, dtype=tf.float32)),  # Input 3
        tf.TensorSpec(shape=(num_classes,), dtype=tf.float32)  # Labels
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(x_test, y_test),
    output_signature=(
        (tf.TensorSpec(shape=input_shape, dtype=tf.float32),  # Input 1
         tf.TensorSpec(shape=input_shape, dtype=tf.float32),  # Input 2
         tf.TensorSpec(shape=input_shape, dtype=tf.float32)),  # Input 3
        tf.TensorSpec(shape=(num_classes,), dtype=tf.float32)  # Labels
    )
)

# Batch and optimize performance
train_multi_input = train_dataset.batch(64).prefetch(tf.data.AUTOTUNE)
val_multi_input = val_dataset.batch(64).prefetch(tf.data.AUTOTUNE)


In [ ]:
for images, labels in train_multi_input.take(1):
    print(f"Texture Input Shape: {images[0].shape}")  # Should be (batch_size, 224, 224, 3)
    print(f"Color Input Shape: {images[1].shape}")    # Should be (batch_size, 224, 224, 3)
    print(f"Structure Input Shape: {images[2].shape}") # Should be (batch_size, 224, 224, 3)


Texture Input Shape: (64, 224, 224, 3)
Color Input Shape: (64, 224, 224, 3)
Structure Input Shape: (64, 224, 224, 3)


model

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Stop training when validation loss stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reduce learning rate if validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Save the best model based on validation loss
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True)

# Callback List
callbacks = [early_stopping, reduce_lr, checkpoint]


In [ ]:
loss, accuracy, precision, recall, f1_score = model.evaluate(val_generator)
print(f'test Loss: {loss}, test Accuracy: {accuracy}')

In [ ]:
from transformers import TFAutoModel
import tensorflow as tf

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Dense, Dropout, Flatten, Concatenate, GlobalAveragePooling2D,
    Lambda, MultiHeadAttention
)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB4
from transformers import TFAutoModel, AutoConfig

# ✅ Enable Mixed Precision Training to Reduce Memory Usage
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# ✅ Reduce batch size to avoid OOM errors
BATCH_SIZE = 2  # Reduce from 32 to prevent memory overflow

# ✅ Ensure TensorFlow runs functions in Eager Mode
tf.config.run_functions_eagerly(True)

# ============================
# Model Definition
# ============================

# Texture Branch: EfficientNetB4
texture_input = Input(shape=(224, 224, 3), name="texture_input")
texture_base = EfficientNetB4(include_top=False, weights="imagenet")(texture_input)
texture_x = GlobalAveragePooling2D(name="texture_gap")(texture_base)
texture_x = Dense(256, activation="relu", dtype=tf.float32, name="texture_dense")(texture_x)

# Color Branch: Vision Transformer (ViT)
color_input = Input(shape=(224, 224, 3), name="color_input")

# ✅ Fix: Load ViT with correct input configuration
vit_config = AutoConfig.from_pretrained("google/vit-base-patch16-224-in21k")
vit_config.num_channels = 3  # Ensure it expects RGB input

vit_model = TFAutoModel.from_pretrained("google/vit-base-patch16-224-in21k", config=vit_config)

# ✅ Fix: Transpose input from NHWC to NCHW format for ViT
color_input_transposed = Lambda(
    lambda x: tf.transpose(x, perm=[0, 3, 1, 2]),  # Convert NHWC → NCHW
    name="transpose_vit_input"
)(color_input)

# ✅ Enable Gradient Checkpointing to save memory
vit_model.gradient_checkpointing = True

color_features = Lambda(
    lambda x: tf.reshape(vit_model(x, training=False).pooler_output, (-1, 768)),
    output_shape=(768,),
    name="vit_features"
)(color_input_transposed)

color_x = Dense(256, activation="relu", dtype=tf.float32, name="color_dense")(color_features)

# Structure Branch: Swin Transformer
structure_input = Input(shape=(224, 224, 3), name="structure_input")

# ✅ Fix: Load Swin Transformer with correct configuration
swin_config = AutoConfig.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
swin_config.num_channels = 3  # Ensure it expects RGB input

swin_model = TFAutoModel.from_pretrained("microsoft/swin-tiny-patch4-window7-224", config=swin_config)

# ✅ Enable Gradient Checkpointing for Swin Transformer
swin_model.gradient_checkpointing = True

# ✅ Fix: Transpose input from NHWC to NCHW format for Swin Transformer
structure_input_transposed = Lambda(
    lambda x: tf.transpose(x, perm=[0, 3, 1, 2]),  # Convert NHWC → NCHW
    name="transpose_swin_input"
)(structure_input)

structure_features = Lambda(
    lambda x: tf.reshape(swin_model(x, training=False).pooler_output, (-1, 768)),
    output_shape=(768,),
    name="swin_features"
)(structure_input_transposed)

structure_x = Dense(256, activation="relu", dtype=tf.float32, name="structure_dense")(structure_features)

# Fusion Block: Multi-Head Attention
combined_features = Concatenate(name="combined_features")([texture_x, color_x, structure_x])
expanded_features = Lambda(lambda x: tf.expand_dims(x, axis=1),
                           output_shape=lambda s: (s[0], 1, s[1]),
                           name="expand_dims")(combined_features)

attention_features = MultiHeadAttention(num_heads=4, key_dim=192, dtype=tf.float32, name="multi_head_attention")(
    query=expanded_features,
    value=expanded_features,
    key=expanded_features
)

squeezed_features = Lambda(lambda x: tf.squeeze(x, axis=1),
                           output_shape=lambda s: (s[0], s[-1]),
                           name="squeeze_dims")(attention_features)

fusion_output = squeezed_features

# Classification Head
x = Flatten(name="flatten")(fusion_output)
x = Dropout(0.4, name="dropout_1")(x)
x = Dense(256, activation="relu", dtype=tf.float32, name="dense_1")(x)
x = Dropout(0.2, name="dropout_2")(x)
output = Dense(30, activation="softmax", dtype=tf.float32, name="predictions")(x)

# Build & Compile Model
model = Model(inputs=[texture_input, color_input, structure_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Print Model Summary
model.summary()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFSwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFSwinModel

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ color_input (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ structure_input           │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ texture_input             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_2 (Cast)             │ (None, 224, 224, 3)    │              0 │ color_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_4 (Cast)             │ (None, 224, 224, 3)    │              0 │ structure_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ efficientnetb4            │ (None, 7, 7, 1792)     │     17,673,823 │ texture_input[0][0]    │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transpose_vit_input       │ (None, 3, 224, 224)    │              0 │ cast_2[0][0]           │
│ (Lambda)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transpose_swin_input      │ (None, 3, 224, 224)    │              0 │ cast_4[0][0]           │
│ (Lambda)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ texture_gap               │ (None, 1792)           │              0 │ efficientnetb4[0][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ vit_features (Lambda)     │ (None, 768)            │              0 │ transpose_vit_input[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swin_features (Lambda)    │ (None, 768)            │              0 │ transpose_swin_input[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_1 (Cast)             │ (None, 1792)           │              0 │ texture_gap[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_3 (Cast)             │ (None, 768)            │              0 │ vit_features[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_5 (Cast)             │ (None, 768)            │              0 │ swin_features[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ texture_dense (Dense)     │ (None, 256)            │        459,008 │ cast_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ color_dense (Dense)       │ (None, 256)            │        196,864 │ cast_3[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ structure_dense (Dens

 Total params: 21,093,501 (80.47 MB)

 Trainable params: 20,968,294 (79.99 MB)

 Non-trainable params: 125,207 (489.09 KB)

In [10]:
history = model.fit(
    train_multi_input,
    validation_data=val_multi_input,
    epochs=50,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)


NameError: name 'early_stopping' is not defined

In [8]:
def prepare_multi_input(image, label):
    # Duplicate the same image for each branch
    return (image, image, image), label

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_multi_input = train_dataset.map(prepare_multi_input).batch(2).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_multi_input = val_dataset.map(prepare_multi_input).batch(2).prefetch(tf.data.AUTOTUNE)


In [13]:
import gc
import tensorflow as tf
from numba import cuda

# Clear Keras session
tf.keras.backend.clear_session()

# Garbage collect
gc.collect()

# Reset CUDA memory
cuda.select_device(0)
cuda.close()
